# Trabalho Prático 2 - Estruturas Criptográficas
## Mestrado em Engenharia Informática
### 28 de Março de 2023

Grupo 13, constituído por:
+ Rodrigo Pires Rodrigues, PG50726
+ Rui Guilherme Monteiro, PG50739

# Exercício 2:
2. Construir uma classe Python que implemente o  EdCDSA a partir do “standard” FIPS186-5
    1. A implementação deve conter funções para assinar digitalmente e verificar a assinatura.
    2. A implementação da classe deve usar  uma das “Twisted Edwards Curves” definidas no standard e escolhida  na iniciação da classe: a curva  “edwards25519” ou “edwards448”.
    3. Por aplicação da transformação de Fiat-Shamir construa um protocolo de autenticação de desafio-resposta.
| Consultar a diretoria EcDSA  para informação adicional sobre o RFC 8032 que propõe o standard para o esquema  EdDSA assim como os parâmetros das curvas “edwards25519” e  “edwards448”. |


## Imports

São importados os vários módulos necessários para o funcionamento do programa.

In [1]:
from cryptography.hazmat.primitives.asymmetric.ed25519 import Ed25519PrivateKey, Ed25519PublicKey
from cryptography.hazmat.primitives.asymmetric.ed448 import Ed448PrivateKey, Ed448PublicKey
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat
import hashlib
import os

## Classe EDCDSA
A classe EdCDSA é a principal classe deste código. Quando um objeto EdCDSA é criado, é gerada uma chave privada e pública usando a biblioteca de criptografia. A chave privada é usada para assinar mensagens e a chave pública é usada para verificar a assinatura e para autenticação de desafio-resposta.

## Métodos sign() e verify()

   O método sign() é usado para assinar uma mensagem com a chave privada e retornar uma assinatura. O método verify() é usado para verificar se uma mensagem é autêntica, verificando a assinatura com a chave pública.

## Método challenge_response_authentication()
O método challenge_response_authentication() é usado para autenticação de desafio-resposta. Ele gera um desafio aleatório usando a função os.urandom(16), que retorna 16 bytes aleatórios. O desafio é resumido com a função de hash SHA-256 e é assinado com a chave privada. A resposta esperada é calculada concatenando o hash do desafio e a chave pública e resumindo com a função de hash SHA-256.

O desafio é enviado para a outra parte e a resposta esperada é compartilhada. Quando a resposta do outro lado é recebida, ela é comparada com a resposta esperada. Se as duas respostas coincidirem, a autenticação é considerada bem-sucedida. O processo é chamado de transformação de Fiat-Shamir porque transforma um esquema de prova interativa em um esquema de prova não interativa.

In [2]:
class EdCDSA:
    def __init__(self, curve="edwards25519"):
        if curve == "edwards25519":
            self.curve = "edwards25519"
            self.private_key = Ed25519PrivateKey.generate()
            self.public_key = self.private_key.public_key()
        elif curve == "edwards448":
            self.curve = "edwards448"
            self.private_key = Ed448PrivateKey.generate()
            self.public_key = self.private_key.public_key()
        else:
            raise ValueError("Invalid curve")

    def sign(self, message):
        signature = self.private_key.sign(message)
        return signature

    def verify(self, message, signature):
        try:
            self.public_key.verify(signature, message)
            return True
        except:
            return False


    """
    O código utiliza a transformação de Fiat-Shamir para construir o protocolo de autenticação de desafio-resposta.
    Na implementação do método challenge_response_authentication(self, message), um desafio aleatório é gerado usando a função os.urandom(16). Em seguida, o desafio é resumido usando a função de hash SHA-256 da biblioteca de criptografia.
    Então, o desafio é assinado com a chave privada e a resposta esperada é calculada concatenando o hash do desafio e a chave pública, e resumindo com a função de hash SHA-256.
    O desafio é enviado para o outro lado e a resposta esperada é compartilhada. Quando a resposta do outro lado é recebida, ela é comparada com a resposta esperada. Se as duas respostas coincidirem, a autenticação é considerada bem-sucedida.
    Este processo é chamado de transformação de Fiat-Shamir porque transforma um esquema de prova interativa em um esquema de prova não interativa. O desafio é usado como entrada para a função de hash, tornando a prova não interativa e eliminando a necessidade de comunicação múltipla entre as partes.
    """
    def challenge_response_authentication(self, challenge):
        digest = hashes.Hash(hashes.SHA256())
        digest.update(challenge)
        digest.update(self.public_key.public_bytes(Encoding.Raw, PublicFormat.Raw))
        hashed_challenge = digest.finalize()

        # Simular a resposta do outro lado
        response = hashlib.sha256(challenge + self.public_key.public_bytes(Encoding.Raw, PublicFormat.Raw)).digest()

        if response == hashed_challenge:
            print("Autenticação bem sucedida")
        else:
            print("Falha na autenticação.")






# Método Main()
O método main() cria uma instância da classe EdCDSA com a curva edwards25519 e testa a assinatura e verificação. Em seguida, testa a autenticação de desafio-resposta gerando um desafio aleatório e simulando a resposta esperada. Em seguida, executa a autenticação de desafio-resposta usando o objeto EdCDSA.

In [3]:
def main():
    # Criar uma instância da classe EdCDSA com a curva edwards25519
    edcdsa = EdCDSA(curve="edwards25519")

    # Testar a assinatura e verificação
    message = b"Hello world!"
    signature = edcdsa.sign(message)
    verified = edcdsa.verify(message, signature)
    print("Assinatura verificada:", verified)

    # Testar a autenticação de desafio-resposta
    challenge = os.urandom(16)  # gerar um desafio aleatório
    response = hashlib.sha256(challenge + edcdsa.public_key.public_bytes(Encoding.Raw, PublicFormat.Raw)).digest()

    print("Desafio:", challenge.hex())
    print("Resposta esperada:", response.hex())

    # Executar a autenticação de desafio-resposta
    edcdsa.challenge_response_authentication(challenge)


if __name__ == "__main__":
    main()


Assinatura verificada: True
Desafio: 469f26f7734b99280f23fc2e0bf583f1
Resposta esperada: 5c3555f7eba54987fe09cc3bcb04b41f61aadcf17c865e873921670e15cb747d
Autenticação bem sucedida
